# Generating structured mesh with numpy

In [5]:
import argiope as ag
import numpy as np
import pandas as pd

In [32]:
Nex, Ney = 2, 3 # Element numbers
lx, ly = 1., 1. # Length
nlabels = np.arange((Nex + 1) * (Ney +1)) + 1
elabels = np.arange(Nex * Ney) + 1

In [33]:
x = np.linspace(0., lx, Nex + 1)
y = np.linspace(0., lx, Ney + 1)
X, Y = np.meshgrid(x, y)
X

array([[ 0. ,  0.5,  1. ],
       [ 0. ,  0.5,  1. ],
       [ 0. ,  0.5,  1. ],
       [ 0. ,  0.5,  1. ]])

In [34]:
Y

array([[ 0.        ,  0.        ,  0.        ],
       [ 0.33333333,  0.33333333,  0.33333333],
       [ 0.66666667,  0.66666667,  0.66666667],
       [ 1.        ,  1.        ,  1.        ]])

In [35]:
nlabels = nlabels.reshape(Nex+1, Ney+1)
nlabels

array([[ 1,  2,  3,  4],
       [ 5,  6,  7,  8],
       [ 9, 10, 11, 12]])

In [36]:
elabels = elabels.reshape(Nex, Ney)
elabels

array([[1, 2, 3],
       [4, 5, 6]])

In [37]:
conn = []
for i in range(Nex):
    for j in range(Ney):
        conn.append( nlabels[i:i+2, j:j+2].flatten() ) 
conn = np.array(conn)
conn

array([[ 1,  2,  5,  6],
       [ 2,  3,  6,  7],
       [ 3,  4,  7,  8],
       [ 5,  6,  9, 10],
       [ 6,  7, 10, 11],
       [ 7,  8, 11, 12]])

In [38]:
nl = nlabels
np.array([
    nl[  :-1,  :-1],
    nl[  :-1, 1:  ],
    nl[ 1:  , 1:  ],
    nl[ 1:  ,  :-1],
    ]).swapaxes(0,2)

array([[[ 1,  2,  6,  5],
        [ 5,  6, 10,  9]],

       [[ 2,  3,  7,  6],
        [ 6,  7, 11, 10]],

       [[ 3,  4,  8,  7],
        [ 7,  8, 12, 11]]])